In [39]:
import boto3
import pandas as pd
import time
from io import StringIO #BytesIO

In [40]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "us-east-1"
SCHEMA_NAME = "covid_dataset"
S3_STAGING_DIR = "s3://pngo-covid-athena-output/output/"
S3_BUCKET_NAME = "pngo-covid-athena-output"
S3_OUTPUT_DIR= "output"

In [41]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = AWS_REGION,
)

In [42]:
Dict = {}

#download function 
def download_and_load_query_results (
        client:boto3.client, 
        query_response:Dict
)-> pd.DataFrame:
    while True:
        try: 
            # client.get_query_execution(QueryExecutionId = query_response["QueryExecutionId"])
            client.get_query_results(QueryExecutionId = query_response["QueryExecutionId"])
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
            
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_KEY,
        region_name = AWS_REGION,
    )

    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location) 

In [44]:
#enigma_jhud download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhud limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

engima_jhud = download_and_load_query_results(athena_client, response)
engima_jhud

ClientError: An error occurred (UnrecognizedClientException) when calling the StartQueryExecution operation: The security token included in the request is invalid.

In [45]:
#us_county download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_county limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_county = download_and_load_query_results(athena_client, response)

ClientError: An error occurred (UnrecognizedClientException) when calling the StartQueryExecution operation: The security token included in the request is invalid.

In [ ]:
#us_states download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_states limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_states = download_and_load_query_results(athena_client, response)

In [ ]:
#countrycode download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM countrycode limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

countrycode = download_and_load_query_results(athena_client, response)

In [ ]:
#countypopulation download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM countypopulation limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

countypopulation = download_and_load_query_results(athena_client, response)

In [ ]:
#rearc_usa_hospital_beds download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM rearc_usa_hospital_beds limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)

In [ ]:
#state_abv download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM state_abv limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

state_abv = download_and_load_query_results(athena_client, response)
state_abv.head()

In [ ]:
#states_daily download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM states_daily limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

states_daily = download_and_load_query_results(athena_client, response)
states_daily.head()

In [ ]:
#us_daily download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_daily limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_daily = download_and_load_query_results(athena_client, response)

In [ ]:
#us_total_latest download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_total_latest limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_total_latest = download_and_load_query_results(athena_client, response)

In [ ]:
# Cleaning data

new_header = state_abv.iloc[0]
state_abv = state_abv[1:]
state_abv.columns = new_header
state_abv.head()


In [ ]:
## Extract columns from staging tables

# Fact table

factCovid_1 = engima_jhud[[
    'fips','province_state','country_region','confirmed','deaths','recovered','active'
]]
factCovid_1


In [ ]:
factCovid_2 = states_daily[[
    'fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged'
]]
factCovid_2

In [ ]:
# Merging output table

factCovid = pd.merge(factCovid_1, factCovid_2, on = 'fips', how = 'outer')
factCovid

In [ ]:
# Dimension tables

dimRegion_1 = engima_jhud[[
    'fips','province_state','country_region', 'latitude','longitude'
]]

dimRegion_2 = us_county[[
    'fips','county','state'
]]

dimRegion = pd.merge(dimRegion_1, dimRegion_2, on = 'fips', how = 'inner')